In [1]:
import os
import numpy as np
import random
import theano
from pandas.io.parsers import read_csv
from sklearn.utils import shuffle
from scipy import misc
import pandas as pd
from matplotlib import pyplot
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss

Using gpu device 0: GeForce GTX 760


In [2]:
import matplotlib.pyplot as plt
 
pd.set_option('display.max_columns',1000)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 100)
 
plt.style.use('ggplot')

#Chargement des data MNIST (cf Kaggle)

In [3]:
from preprocess_data_lib import *
(X_train, y_train, X_valid, y_valid, X_test, y_test) = getDataRot(cast=True)

In [4]:
(X_train.shape, y_train.shape, X_valid.shape, y_valid.shape, X_test.shape, y_test.shape)

((126000, 784), (126000,), (0, 784), (0,), (28000, 784), (28000,))

In [5]:
image_width = image_height = int(X_train.shape[1] ** .5)

In [6]:
X_train_reshaped = X_train.reshape(X_train.shape[0], 1, image_height, image_width)
X_test_reshaped = X_test.reshape(X_test.shape[0], 1, image_height, image_width)

In [7]:
from lasagne import layers
from lasagne.nonlinearities import  softmax, rectify
from lasagne.updates import nesterov_momentum
from nolearn.lasagne import NeuralNet

##On prépare la convolution

In [8]:
def float32(k):
    return np.cast['float32'](k)

In [9]:
class AdjustVariable(object):
    def __init__(self, name, start=0.03, stop=0.001):
        self.name = name
        self.start, self.stop = start, stop
        self.ls = None

    def __call__(self, nn, train_history):
        if self.ls is None:
            self.ls = np.linspace(self.start, self.stop, nn.max_epochs)

        epoch = train_history[-1]['epoch']
        new_value = float32(self.ls[epoch - 1])
        getattr(nn, self.name).set_value(new_value)

In [10]:
ffNet1 = NeuralNet(
    layers=[  # three layers: one hidden layer
        ('input', layers.InputLayer),
        ('hidden', layers.DenseLayer),
        ('output', layers.DenseLayer),
        ],
    # layer parameters:
    input_shape=(None, X_train.shape[1]),  # 96x96 input pixels per batch
    hidden_num_units=20,  # number of units in hidden layer
    output_nonlinearity=softmax,  # output layer uses identity function
    output_num_units=10,  # 30 target values

    # optimization method:
    update=nesterov_momentum,
    update_learning_rate=theano.shared(float32(0.03)),
    update_momentum=theano.shared(float32(0.9)),
    
    regression=False,
    max_epochs=5,  # we want to train this many epochs
    verbose=1,
    
    on_epoch_finished=[
        AdjustVariable('update_learning_rate', start=0.03, stop=0.0001),
        AdjustVariable('update_momentum', start=0.9, stop=0.999),
        ],
    
    )
ffNet1.fit(X_train, y_train)

  input             	(None, 784)         	produces     784 outputs
  hidden            	(None, 20)          	produces      20 outputs
  output            	(None, 10)          	produces      10 outputs
|   epoch |   train loss |   valid loss |   valid best |   train/val |   val acc |    dur |
|--------:|-------------:|-------------:|-------------:|------------:|----------:|-------:|
|       1 |       0.4410 |       0.3117 |       0.3117 |      1.4148 |    0.9073 | 0.8474 |
|       2 |       0.2698 |       0.2656 |       0.2656 |      1.0158 |    0.9211 | 0.8548 |
|       3 |       0.2393 |       0.2441 |       0.2441 |      0.9805 |    0.9276 | 0.8459 |
|       4 |       0.2225 |       0.2362 |       0.2362 |      0.9421 |    0.9288 | 0.8510 |
|       5 |       0.2115 |       0.2339 |       0.2339 |      0.9040 |    0.9286 | 0.8487 |


/home/issam/anaconda/lib/python2.7/site-packages/Lasagne-0.1.dev0-py2.7.egg/lasagne/init.py:86: UserWarning: The uniform initializer no longer uses Glorot et al.'s approach to determine the bounds, but defaults to the range (-0.01, 0.01) instead. Please use the new GlorotUniform initializer to get the old behavior. GlorotUniform is now the default for all layers.
  warnings.warn("The uniform initializer no longer uses Glorot et al.'s "


NeuralNet(X_tensor_type=<function matrix at 0x7f31503d0320>,
     batch_iterator_test=<nolearn.lasagne.base.BatchIterator object at 0x7f313618b390>,
     batch_iterator_train=<nolearn.lasagne.base.BatchIterator object at 0x7f313618b850>,
     custom_score=None, eval_size=0.2, hidden_num_units=20,
     input_shape=(None, 784),
     layers=[('input', <class 'lasagne.layers.input.InputLayer'>), ('hidden', <class 'lasagne.layers.dense.DenseLayer'>), ('output', <class 'lasagne.layers.dense.DenseLayer'>)],
     loss=None, max_epochs=5, more_params={},
     objective=<class 'lasagne.objectives.Objective'>,
     objective_loss_function=<function categorical_crossentropy at 0x7f314ee94758>,
     on_epoch_finished=[<__main__.AdjustVariable object at 0x7f313d319990>, <__main__.AdjustVariable object at 0x7f3135f99350>],
     on_training_finished=(),
     output_nonlinearity=<function softmax at 0x7f3137aff668>,
     output_num_units=10, regression=False,
     update=<function nesterov_momentum at 

In [12]:
ffNet2 = NeuralNet(
        layers=[  #list the layers here
            ('input', layers.InputLayer),
            ('hidden1', layers.DenseLayer),
            ('hidden2', layers.DenseLayer),
            ('output', layers.DenseLayer),
            ],

        # layer parameters:
        input_shape=(None, X_train.shape[1]),
        hidden1_num_units=200, hidden1_nonlinearity=rectify,  #params of first layer
        hidden2_num_units=200, hidden2_nonlinearity=rectify,
        output_nonlinearity=softmax,  # softmax for classification problems
        output_num_units=10,  # 10 target values

        # optimization method:
        update=nesterov_momentum,
        update_learning_rate=theano.shared(float32(0.05)),
        update_momentum=theano.shared(float32(0.7)),

        on_epoch_finished=[
            AdjustVariable('update_learning_rate', start=0.05, stop=0.0001),
            AdjustVariable('update_momentum', start=0.7, stop=0.999),
            ],    
    
        regression=False,
        max_epochs=50,  # Intentionally limited for execution speed
        verbose=1,
        )
ffNet2.fit(X_train, y_train)

  input             	(None, 784)         	produces     784 outputs
  hidden1           	(None, 200)         	produces     200 outputs
  hidden2           	(None, 200)         	produces     200 outputs
  output            	(None, 10)          	produces      10 outputs
|   epoch |   train loss |   valid loss |   valid best |   train/val |   val acc |    dur |
|--------:|-------------:|-------------:|-------------:|------------:|----------:|-------:|
|       1 |       0.3437 |       0.1936 |       0.1936 |      1.7751 |    0.9414 | 1.3447 |
|       2 |       0.1432 |       0.1343 |       0.1343 |      1.0662 |    0.9586 | 1.3033 |
|       3 |       0.1009 |       0.1124 |       0.1124 |      0.8972 |    0.9659 | 1.3002 |
|       4 |       0.0789 |       0.1007 |       0.1007 |      0.7834 |    0.9704 | 1.3019 |
|       5 |       0.0641 |       0.0939 |       0.0939 |      0.6820 |    0.9716 | 1.3038 |
|       6 |       0.0528 |       0.0895 |       0.0895 |      0.5903 |    0.9726 | 1.304

NeuralNet(X_tensor_type=<function matrix at 0x7f31503d0320>,
     batch_iterator_test=<nolearn.lasagne.base.BatchIterator object at 0x7f313618b390>,
     batch_iterator_train=<nolearn.lasagne.base.BatchIterator object at 0x7f313618b850>,
     custom_score=None, eval_size=0.2,
     hidden1_nonlinearity=<function rectify at 0x7f3137aff758>,
     hidden1_num_units=200,
     hidden2_nonlinearity=<function rectify at 0x7f3137aff758>,
     hidden2_num_units=200, input_shape=(None, 784),
     layers=[('input', <class 'lasagne.layers.input.InputLayer'>), ('hidden1', <class 'lasagne.layers.dense.DenseLayer'>), ('hidden2', <class 'lasagne.layers.dense.DenseLayer'>), ('output', <class 'lasagne.layers.dense.DenseLayer'>)],
     loss=None, max_epochs=50, more_params={},
     objective=<class 'lasagne.objectives.Objective'>,
     objective_loss_function=<function categorical_crossentropy at 0x7f314ee94758>,
     on_epoch_finished=[<__main__.AdjustVariable object at 0x7f3135f99ed0>, <__main__.AdjustV

In [13]:
convNet1 = NeuralNet(
    layers=[
        ('input', layers.InputLayer),
        ('conv1', layers.Conv2DLayer),      #Convolutional layer.  Params defined below
        ('pool1', layers.MaxPool2DLayer),   #Like downsampling, for execution speed
        ('conv2', layers.Conv2DLayer),
        ('hidden3', layers.DenseLayer),
        ('output', layers.DenseLayer),
        ],

        input_shape=(None, 1, image_width, image_height),
        conv1_num_filters=7, conv1_filter_size=(3, 3), conv1_nonlinearity=rectify,
        pool1_pool_size=(3, 3),
        conv2_num_filters=12, conv2_filter_size=(2, 2), conv2_nonlinearity=rectify,
        hidden3_num_units=50,
        output_num_units=10, output_nonlinearity=softmax,
    
        update_learning_rate=theano.shared(float32(0.05)),
        update_momentum=theano.shared(float32(0.7)),

        on_epoch_finished=[
            AdjustVariable('update_learning_rate', start=0.05, stop=0.0001),
            AdjustVariable('update_momentum', start=0.7, stop=0.999),
            ],     
    
        regression=False,
        max_epochs=5,
        verbose=1,
)
convNet1.fit(X_train_reshaped, y_train)

  input             	(None, 1, 28, 28)   	produces     784 outputs
  conv1             	(None, 7, 26, 26)   	produces    4732 outputs
  pool1             	(None, 7, 9, 9)     	produces     567 outputs
  conv2             	(None, 12, 8, 8)    	produces     768 outputs
  hidden3           	(None, 50)          	produces      50 outputs
  output            	(None, 10)          	produces      10 outputs
|   epoch |   train loss |   valid loss |   valid best |   train/val |   val acc |     dur |
|--------:|-------------:|-------------:|-------------:|------------:|----------:|--------:|
|       1 |       0.2966 |       0.1125 |       0.1125 |      2.6353 |    0.9652 | 11.1504 |
|       2 |       0.0972 |       0.0802 |       0.0802 |      1.2110 |    0.9747 | 11.1565 |
|       3 |       0.0697 |       0.0742 |       0.0742 |      0.9387 |    0.9760 | 11.1568 |
|       4 |       0.0567 |       0.0698 |       0.0698 |      0.8119 |    0.9779 | 11.4567 |
|       5 |       0.0491 |       0.0666 

NeuralNet(X_tensor_type=<function tensor4 at 0x7f31503d06e0>,
     batch_iterator_test=<nolearn.lasagne.base.BatchIterator object at 0x7f313618b390>,
     batch_iterator_train=<nolearn.lasagne.base.BatchIterator object at 0x7f313618b850>,
     conv1_filter_size=(3, 3),
     conv1_nonlinearity=<function rectify at 0x7f3137aff758>,
     conv1_num_filters=7, conv2_filter_size=(2, 2),
     conv2_nonlinearity=<function rectify at 0x7f3137aff758>,
     conv2_num_filters=12, custom_score=None, eval_size=0.2,
     hidden3_num_units=50, input_shape=(None, 1, 28, 28),
     layers=[('input', <class 'lasagne.layers.input.InputLayer'>), ('conv1', <class 'lasagne.layers.conv.Conv2DLayer'>), ('pool1', <class 'lasagne.layers.pool.MaxPool2DLayer'>), ('conv2', <class 'lasagne.layers.conv.Conv2DLayer'>), ('hidden3', <class 'lasagne.layers.dense.DenseLayer'>), ('output', <class 'lasagne.layers.dense.DenseLayer'>)],
     loss=None, max_epochs=5, more_params={},
     objective=<class 'lasagne.objectives.Obj

In [14]:
convNet11 = NeuralNet(
    layers=[
        ('input', layers.InputLayer),
        ('conv1', layers.Conv2DLayer),
        ('pool1', layers.MaxPool2DLayer),
        ('dropout1', layers.DropoutLayer),  # !
        ('conv2', layers.Conv2DLayer),
        ('pool2', layers.MaxPool2DLayer),
        ('dropout2', layers.DropoutLayer),  # !
        ('hidden4', layers.DenseLayer),
        ('hidden5', layers.DenseLayer),
        ('output', layers.DenseLayer),
        ],
    input_shape=(None, 1, image_width, image_height),
    conv1_num_filters=4, conv1_filter_size=(4, 4), conv1_nonlinearity=rectify,
    pool1_pool_size=(3, 3),
    dropout1_p=0.1,  # !
    conv2_num_filters=8, conv2_filter_size=(3, 3), conv2_nonlinearity=rectify,
    pool2_pool_size=(2, 2),
    dropout2_p=0.1,  # !
    hidden4_num_units=500, 
    hidden5_num_units=500,
    output_num_units=10, output_nonlinearity=softmax,

    update=nesterov_momentum,
    
    update_learning_rate=theano.shared(float32(0.07)),
    update_momentum=theano.shared(float32(0.4)),

    on_epoch_finished=[
        AdjustVariable('update_learning_rate', start=0.07, stop=0.0001),
        AdjustVariable('update_momentum', start=0.4, stop=0.999),
        ],  
    
    regression=False,
    max_epochs=25,
    verbose=1,
    )
convNet11.fit(X_train_reshaped, y_train)

  input             	(None, 1, 28, 28)   	produces     784 outputs
  conv1             	(None, 4, 25, 25)   	produces    2500 outputs
  pool1             	(None, 4, 9, 9)     	produces     324 outputs
  dropout1          	(None, 4, 9, 9)     	produces     324 outputs
  conv2             	(None, 8, 7, 7)     	produces     392 outputs
  pool2             	(None, 8, 4, 4)     	produces     128 outputs
  dropout2          	(None, 8, 4, 4)     	produces     128 outputs
  hidden4           	(None, 500)         	produces     500 outputs
  hidden5           	(None, 500)         	produces     500 outputs
  output            	(None, 10)          	produces      10 outputs
|   epoch |   train loss |   valid loss |   valid best |   train/val |   val acc |     dur |
|--------:|-------------:|-------------:|-------------:|------------:|----------:|--------:|
|       1 |       0.4632 |       0.1273 |       0.1273 |      3.6377 |    0.9594 | 12.6152 |
|       2 |       0.1562 |       0.0811 |       0.0

NeuralNet(X_tensor_type=<function tensor4 at 0x7f31503d06e0>,
     batch_iterator_test=<nolearn.lasagne.base.BatchIterator object at 0x7f313618b390>,
     batch_iterator_train=<nolearn.lasagne.base.BatchIterator object at 0x7f313618b850>,
     conv1_filter_size=(4, 4),
     conv1_nonlinearity=<function rectify at 0x7f3137aff758>,
     conv1_num_filters=4, conv2_filter_size=(3, 3),
     conv2_nonlinearity=<function rectify at 0x7f3137aff758>,
     conv2_num_filters=8, custom_score=None, dropout1_p=0.1,
     dropout2_p=0.1, eval_size=0.2, hidden4_num_units=500,
     hidden5_num_units=500, input_shape=(None, 1, 28, 28),
     layers=[('input', <class 'lasagne.layers.input.InputLayer'>), ('conv1', <class 'lasagne.layers.conv.Conv2DLayer'>), ('pool1', <class 'lasagne.layers.pool.MaxPool2DLayer'>), ('dropout1', <class 'lasagne.layers.noise.DropoutLayer'>), ('conv2', <class 'lasagne.layers.conv.Conv2DLayer'>), ('pool2', <cla..., <class 'lasagne.layers.dense.DenseLayer'>), ('output', <class 'la

In [15]:
y_preds = convNet11.predict(X_test_reshaped)

In [16]:
pd.DataFrame(y_preds).to_csv('../../data/intermediate/no_trainRot_9889_pred.csv', index=None, header=None)

In [14]:
convNet2 = NeuralNet(
    layers=[
        ('input', layers.InputLayer),
        ('conv1', layers.Conv2DLayer),
        ('pool1', layers.MaxPool2DLayer),
        ('dropout1', layers.DropoutLayer),  # !
        ('conv2', layers.Conv2DLayer),
        ('pool2', layers.MaxPool2DLayer),
        ('dropout2', layers.DropoutLayer),  # !
        ('conv3', layers.Conv2DLayer),
        ('pool3', layers.MaxPool2DLayer),
        ('dropout3', layers.DropoutLayer),  # !
        ('hidden4', layers.DenseLayer),
        ('hidden5', layers.DenseLayer),
        ('output', layers.DenseLayer),
        ],
    input_shape=(None, 1, image_width, image_height),
    conv1_num_filters=4, conv1_filter_size=(4, 4), conv1_nonlinearity=rectify,
    pool1_pool_size=(3, 3),
    dropout1_p=0.1,  # !
    conv2_num_filters=8, conv2_filter_size=(3, 3), conv2_nonlinearity=rectify,
    pool2_pool_size=(2, 2),
    dropout2_p=0.1,  # !
    conv3_num_filters=16, conv3_filter_size=(2, 2), conv3_nonlinearity=rectify,
    pool3_pool_size=(2, 2),
    dropout3_p=0.1,  # !
    hidden4_num_units=100, 
    hidden5_num_units=100,
    output_num_units=10, output_nonlinearity=softmax,

    update=nesterov_momentum,
    
    update_learning_rate=theano.shared(float32(0.07)),
    update_momentum=theano.shared(float32(0.4)),

    on_epoch_finished=[
        AdjustVariable('update_learning_rate', start=0.07, stop=0.0001),
        AdjustVariable('update_momentum', start=0.4, stop=0.999),
        ],  
    
    regression=False,
    max_epochs=50,
    verbose=1,
    )
convNet2.fit(X_train_reshaped, y_train)

  input             	(None, 1, 28, 28)   	produces     784 outputs
  conv1             	(None, 4, 25, 25)   	produces    2500 outputs
  pool1             	(None, 4, 9, 9)     	produces     324 outputs
  dropout1          	(None, 4, 9, 9)     	produces     324 outputs
  conv2             	(None, 8, 7, 7)     	produces     392 outputs
  pool2             	(None, 8, 4, 4)     	produces     128 outputs
  dropout2          	(None, 8, 4, 4)     	produces     128 outputs
  conv3             	(None, 16, 3, 3)    	produces     144 outputs
  pool3             	(None, 16, 2, 2)    	produces      64 outputs
  dropout3          	(None, 16, 2, 2)    	produces      64 outputs
  hidden4           	(None, 100)         	produces     100 outputs
  hidden5           	(None, 100)         	produces     100 outputs
  output            	(None, 10)          	produces      10 outputs
|   epoch |   train loss |   valid loss |   valid best |   train/val |   val acc |    dur |
|--------:|-------------:|-----------

NeuralNet(X_tensor_type=<function tensor4 at 0x7efc0405e6e0>,
     batch_iterator_test=<nolearn.lasagne.base.BatchIterator object at 0x7efbe9d726d0>,
     batch_iterator_train=<nolearn.lasagne.base.BatchIterator object at 0x7efbe9d722d0>,
     conv1_filter_size=(4, 4),
     conv1_nonlinearity=<function rectify at 0x7efbeb794398>,
     conv1_num_filters=4, conv2_filter_size=(3, 3),
     conv2_nonlinearity=<function rectify at 0x7efbeb794398>,
     conv2_num_filters=8, conv3_filter_size=(2, 2),
     conv3_nonlinearity=<function rectify at 0x7efbeb794398>,
     conv3_num_filters=16, custom_score=None, dropout1_p=0.1,
     dropout2_p=0.1, dropout3_p=0.1, eval_size=0.2, hidden4_num_units=100,
     hidden5_num_units=100, input_shape=(None, 1, 28, 28),
     layers=[('input', <class 'lasagne.layers.input.InputLayer'>), ('conv1', <class 'lasagne.layers.conv.Conv2DLayer'>), ('pool1', <class 'lasagne.layers.pool.MaxPool2DLayer'>), ('dropout1', <class 'lasagne.layers.noise.DropoutLayer'>), ('conv2

In [14]:
y_preds = convNet2.predict(X_test_reshaped)

In [15]:
pd.DataFrame(y_preds).to_csv('../../data/intermediate/no_pred.csv', index=None, header=None)